In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
from matplotlib import pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
main_dir = os.path.dirname(os.getcwd())

In [ ]:
main_dir

In [ ]:
faster_rcnn_dir = os.path.join(main_dir, 'models/faster_rcnn/faster_rcnn_inception_v2_coco_2018_01_28')

In [ ]:
# Read the graph.
with tf.gfile.FastGFile( os.path.join(faster_rcnn_dir,'frozen_inference_graph.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [ ]:
sess = tf.Session()

In [ ]:
sess.graph.as_default()
tf.import_graph_def(graph_def, name='')

In [ ]:
darknet_dir = os.path.join(main_dir, 'models/darknet')

In [ ]:
img_path = os.path.join(darknet_dir, "data/horses.jpg")
img_array = cv2.imread(img_path)

In [ ]:
plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
rows = img_array.shape[0]
cols = img_array.shape[1]
inp = img_array[:, :, [2, 1, 0]]  # BGR2RGB


In [ ]:
# Run the model
out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                sess.graph.get_tensor_by_name('detection_scores:0'),
                sess.graph.get_tensor_by_name('detection_boxes:0'),
                sess.graph.get_tensor_by_name('detection_classes:0')],
               feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

In [ ]:
sess.close()

In [ ]:
cvNet = cv2.dnn.readNetFromTensorflow(os.path.join(faster_rcnn_dir,'frozen_inference_graph.pb'), os.path.join(faster_rcnn_dir,'graph.pbtxt'))

In [ ]:
img_path = os.path.join(darknet_dir, "data/horses.jpg")
img_array = cv2.imread(img_path)

In [ ]:
rows = img_array.shape[0]
cols = img_array.shape[1]

In [ ]:
cvNet.setInput(cv2.dnn.blobFromImage(img_array, size=(300, 300), swapRB=True, crop=False))
cvOut = cvNet.forward()

In [ ]:
for detection in cvOut[0,0,:,:]:
    score = float(detection[2])
    if score > 0.3:
        left = detection[3] * cols
        top = detection[4] * rows
        right = detection[5] * cols
        bottom = detection[6] * rows
        cv2.rectangle(img_array, (int(left), int(top)), (int(right), int(bottom)), (23, 230, 210), thickness=2)


In [ ]:
plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
plt.show()